# Understanding AI Prompts: Chain of Thought (CoT) vs. ReAct

This guide explores two techniques for "prompting" Large Language Models (LLMs), which are AI systems trained on vast amounts of text data to understand and generate human-like language. We'll compare "Chain of Thought" (CoT) and "ReAct" (Reason+Act) prompting. Think of prompting as how you ask the AI to do something. The way you frame your request can significantly change the quality and accuracy of its response.

For an Ops professional, understanding these techniques can be valuable when:
* Integrating AI into monitoring or automation scripts.
* Building tools that use AI to summarize logs or incidents.
* Evaluating AI solutions for operational tasks.

### Verifying Python Software Installation (optional)

The following `pip install` command ensures all necessary Python software components (dependencies) are present.

It's safe to run for verification, even if you've already completed Module 02 (which should have handled this).

In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Getting Started: Setting Up Our AI Connection

This first code block is about setting up the connection to an AI model. In a real-world Ops scenario, this might involve connecting to a centrally managed AI service, ensuring credentials are secure, and understanding the endpoint (the address of the AI service). (We've deeply commented the code for those interested.)

In [2]:
# Import necessary Python libraries
import openai         # The official library for interacting with OpenAI models (or compatible APIs)
import re             # Stands for 'regular expression', a tool for pattern matching in text
import httpx          # A modern HTTP client library, used for making web requests (how we talk to the AI model)
import os                # Provides ways to interact with the operating system (e.g., environment variables for API keys)
from rich import print   # A library for creating rich text and beautiful formatting in the terminal (helps make output clearer)
import json              # For working with JSON data, a common format for APIs

# Specifically import the OpenAI client class
from openai import OpenAI

# --- Configuration for Connecting to the AI Model ---
# Ops Perspective: These would typically be managed via configuration files,
# environment variables, or a secrets management system in a production environment.

# API Key: This is like a password that gives us access to the AI model.
# IMPORTANT: Never hardcode real API keys in your scripts. This is a placeholder.
api_key = "placeholder"  

# Model Name: Specifies which AI model we want to use.
# Different models have different capabilities (e.g., speed, accuracy, cost).
# "mistral-small:latest" and "qwen3:32b" are examples of other specific model versions.

model = "llama3.2:3b-instruct-fp16" 

# Base URL: This is the network address where the AI model is hosted.
# In this case, "http://localhost:11434/v1/" suggests it's running locally on your machine
# (perhaps using a tool like Ollama or a local inference server).
# In an enterprise setup, this would be a secure, managed API endpoint.
base_url = "http://localhost:11434/v1/"

# --- Initialize the AI Client ---
# This creates an 'object' (an instance of the OpenAI class) that we'll use
# to send requests to the AI model and get responses.
# It's configured with the URL and API key we defined above.
client = OpenAI(
    base_url=base_url,
    api_key=api_key,
)

# Print a confirmation message
# print("Python libraries imported and AI Client connection configured.")
print("[green] Imports complete, Client initialized, Model setup[/green]")

 Imports complete, Client initialized, Model setup

**Ops Focus & Python/AI for Beginners:**

* **`import` statements:** These lines bring in pre-written code (libraries) that provide useful functions. For example, `openai` helps talk to AI models, and `os` can help manage things like API keys (passwords for services).
* **`api_key`:** This is like a password for the AI service. In real systems, you'd never type it directly into the code. You'd use secure methods like environment variables or a secrets vault.
* **`model`:** AI services offer different "models" (like different versions of software). Some are faster, some are better at specific tasks. Choosing the right one is like picking the right tool for a job.
* **`base_url`:** This is the web address of the AI service. `localhost` means it's running on the same machine as this script. In a company, this would be a central, managed service address.
* **`client = OpenAI(...)`:** This line creates a "client" object. Think of it as setting up a dedicated communication channel to the AI service, using the address and password we specified.

# Chain of Thought (CoT) Prompting

## What is Chain of Thought?

Imagine you're trying to solve a complex problem. You wouldn't just jump to the answer. You'd think it through, step by step. **Chain of Thought (CoT) prompting** is a way to ask an AI model to do the same thing: show its work before giving the final answer.

Instead of just asking:
`"What's 5 + 3 * 2?"` (which could be ambiguous)

You might guide it with CoT:
`"First, tell me the order of operations. Then calculate 3 * 2. Finally, add 5 to that result. What's the final answer?"`

**Why is this useful, especially in Ops?**

* **Better Accuracy for Complex Tasks:** If an AI needs to diagnose a multi-step failure or parse a complex log, forcing it to "think step-by-step" can lead to more accurate results.
* **Transparency & Debugging:** You can see *how* the AI reached its conclusion. If it makes a mistake, you can often pinpoint where its reasoning went wrong. This is crucial for trusting AI in operational tasks.
* **Reduces "Hallucinations":** Sometimes AI models confidently make up incorrect information (called "hallucinations"). CoT can help reduce this by making the reasoning process more explicit.

### Few-Shot Learning (A Quick Detour)

The example below uses a concept called **"few-shot prompting."**

* **Zero-Shot:** You ask the AI a question directly without any examples.
    * `"Translate 'hello' to French."`
* **One-Shot:** You give the AI one example before asking your question.
    * `"English: goodbye -> French: au revoir. Now, English: hello -> French: ?"`
* **Few-Shot:** You give the AI several examples. This helps the AI better understand the pattern, format, or type of reasoning you expect. The CoT example below uses few-shot prompting by providing multiple question/thought/answer examples.

From an Ops perspective, if you're trying to get an AI to classify alerts or summarize incident reports in a specific format, providing a few good examples (few-shot) can drastically improve its performance and consistency.

### CoT Example:

This code shows how to use CoT. We give the AI a "system prompt" which is like setting the rules for how it should behave. Then we ask our actual question.

In [4]:
# This is the "system prompt" - instructions for the AI.
# It tells the AI to always provide a "Thought" process before the "Answer".
# It also includes several examples (few-shot learning) to show the desired format.

cot_system_prompt = """
You are a thoughtful and logical assistant. For every question, you will:
- Think step-by-step under a “Thought” section.
- Then write the final result under “Answer”.
- Always follow the structure shown below.

Use this format:
Question: <the question>
Thought: <your detailed reasoning>
Answer: <final answer>

Here are some examples:

Question: If a train leaves at 2 PM and takes 3 hours to reach its destination, what time does it arrive?
Thought: The train departs at 2 PM. If it travels for 3 hours, it will arrive at 2 + 3 = 5 PM.
Answer: 5 PM

Question: What is the capital of the country whose official language is French and borders Germany?
Thought: France is a country that borders Germany and has French as its official language. The capital of France is Paris.
Answer: Paris

Question: What is the sum of the first three even numbers?
Thought: The first three even numbers are 2, 4, and 6. Their sum is 2 + 4 + 6 = 12.
Answer: 12

Now answer the next question using the same format:
"""

# This is the actual question we want the AI to answer.
user_question_cot = "If Tom has 5 cookies and eats 2, how many does he have left?"

# --- Preparing the request for the AI ---
# AI models often work with a "conversation history" or a list of messages.
# "system" role: Sets the overall behavior of the AI.
# "user" role: Represents what the human user is asking.
messages_for_cot = [
    {"role": "system", "content": cot_system_prompt},
    {"role": "user", "content": user_question_cot}
]

# --- Sending the request to the AI and getting the response ---
# `client.chat.completions.create(...)` is the function call to the AI.
# - `model=model`: Tells it which AI model to use (that we configured earlier).
# - `messages=messages_for_cot`: Passes our structured request.
completion_cot = client.chat.completions.create(
    model=model,
    messages=messages_for_cot,
)

# --- Displaying the AI's response ---
# The AI's answer is typically nested within the 'completion' object.
# `completion_cot.choices[0].message.content` accesses the text of the AI's reply.

print(f"[green] Question: {user_question_cot}[/green] \n")
print(completion_cot.choices[0].message.content)

 Question: If Tom has 5 cookies and eats 2, how many does he have left?

Question: If Tom has 5 cookies and eats 2, how many does he have left?
Thought: To find out how many cookies Tom has left, we need to subtract the number of cookies he ate from the total
number of cookies he had. So, we will subtract 2 (the number of cookies eaten) from 5 (the initial number of 
cookies).
Answer: 3

**Ops Focus & Python/AI for Beginners:**

* **`cot_system_prompt` (String):** This is a multi-line piece of text (a "string" in Python) that acts as the main instruction or persona for the AI. Notice the `"""triple quotes"""` which allow text to span multiple lines.
* **Placeholders like `<the question>`:** These aren't code; they're part of the instructions to the AI, showing it the desired output format.
* **`messages_for_cot` (List of Dictionaries):** This is how we structure the conversation for the AI.
    * A Python `list` is an ordered collection of items, enclosed in `[]`.
    * A Python `dictionary` is a collection of `key: value` pairs, enclosed in `{}`. Here, keys are like `"role"` and `"content"`.
    * `"role": "system"`: Sets the overall context or rules for the AI.
    * `"role": "user"`: Contains the specific question from the user.
* **`client.chat.completions.create(...)`:** This is the command to "run" the AI with our prompt.
* **`completion_cot.choices[0].message.content`:** The AI's response comes back in a structured format. This specific path navigates through that structure to get the actual text of the AI's answer. It might seem complex, but it's a standard way APIs return data. You usually get a primary response (the "choice"), and within that, the message, and then the content of that message.

# ReAct (Reason + Act) Prompting

## What is ReAct?

**ReAct** takes CoT a step further. It stands for **Reason + Act**.

While CoT is about the AI *thinking* step-by-step, ReAct allows the AI to:
1.  **Reason (Thought):** Decide what it needs to know or do.
2.  **Act (Action):** Propose an action to get that information, like using a tool (e.g., a search engine, a calculator, or even an internal company API to check a server's status).
3.  **Observe (Observation):** Get the result from that action.
4.  Repeat this Thought -> Action -> Observation cycle until it has enough information.
5.  **Answer:** Provide the final answer.

**Why is ReAct powerful for Ops?**

* **Interacting with the Real World:** This is key! An Ops AI needs to interact with systems. ReAct provides a framework for the AI to say, "I need to check the current CPU load on server X" (Thought), then specify an action like `execute_command('get_cpu_load serverX')` (Action). Your system would then run this command and feed the result back to the AI (Observation).
* **Using Tools:** You can give the AI "tools" – these could be scripts, API calls, or database queries. For instance:
    * `LookupKnowledgeBase('error code 123')`
    * `GetCurrentOnCall('web_team')`
    * `QueryMetrics('database_latency_p99', 'last_1_hour')`
* **Dynamic Problem Solving:** For problems where the AI needs to gather information incrementally (like troubleshooting a complex outage), ReAct is much more powerful than CoT alone.
* **Transparency:** Like CoT, you see the AI's "thinking" process, including what tools it tried to use and what results it got. This is invaluable for verifying its actions and for auditing.

> ### ReAct vs. React (the JavaScript library)
>
>It's important to note: **ReAct prompting** for LLMs is different from **React**, the popular JavaScript library for building user interfaces. The names are similar, but the concepts are unrelated.

### ReAct Template:

The general flow of a ReAct prompt looks like this:

1.  **Question:** \[The user's initial question]
2.  **Thought:** \[The AI's internal reasoning about what it needs to do next]
3.  **Action:** \[The AI proposes an action, often involving a tool, e.g., `Search('topic')`, `Calculator(2+2)`, `APICall('get_status')`]
4.  **Observation:** \[The result of that action, fed back to the AI]
    * *(...The Thought -> Action -> Observation cycle can repeat multiple times...)*
5.  **Thought:** \[Final reasoning before giving the answer]
6.  **Answer:** \[The AI's final response to the question]

### ReAct Example:

In this example, we simulate the "Action" and "Observation" parts. In a real system, your code would actually *execute* the action (e.g., perform a web search if the action is `Search(...)`) and then provide the real result as the observation.

Here, we are still using **few-shot prompting** by giving an example of the ReAct flow.

In [5]:
# This is a few-shot example demonstrating the ReAct format.
# Notice the Thought -> Action -> Observation flow.
react_few_shot_example = (
    "Question: What is the capital of the country that borders Germany and has Vienna as its capital?\n"
    "Thought: I need to find which country has Vienna as its capital.\n"
    "Action: Lookup('country with capital Vienna')\n"  # The AI suggests an action
    "Observation: Austria\n"  # This would be the *result* of running the Lookup tool
    "Thought: Now I need to check if Austria borders Germany.\n"
    "Action: Lookup('Does Austria border Germany?')\n"
    "Observation: Yes\n"
    "Answer: The capital of Austria, which borders Germany, is Vienna."
)

# A more complex question that might benefit from the ReAct approach
# (requiring the AI to "look up" information in stages).

# Sample question you can experiment with
# user_question_react = "What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?"
# 

user_question_react = "List the primary services and use cases I can deploy with Kubernetes. When finished also output as a list" 

# --- Assembling the Prompt ---
# In this ReAct example, the few-shot example is combined with the new question.
# The `f""` syntax is an "f-string" in Python. It lets you embed variables directly in a string.
# We're telling the AI: "Here's an example of how to think, act, and observe. Now, do the same for this new question."
# The prompt ends with "Thought:" to encourage the AI to start its reasoning process.

react_prompt_for_user = f"""{react_few_shot_example}

Question: {user_question_react}
Thought:""" # We explicitly ask the AI to start with a thought.

# --- Preparing the request for the AI ---
# System Prompt: Sets the overall ReAct behavior.
# User Prompt: Contains the few-shot example and the new question.
messages_for_react = [
    {"role": "system", "content": "You are an assistant that answers questions using a ReAct format: Thought -> Action -> Observation -> Answer. You can use a `Lookup('query')` action to find information."},
    {"role": "user", "content": react_prompt_for_user}
]

# --- Sending the request to the AI ---
# This will take a bit longer because the AI is generating a more complex, multi-step response.
# In a Jupyter Notebook, you'd see a [*] next to the cell while it's processing.
print("[green] Sending ReAct Prompt based request to the LLM. This might take a few moments...\n [/green]")
completion_react = client.chat.completions.create(
    model=model,
    messages=messages_for_react,
)

# --- Displaying the AI's response ---
print(completion_react.choices[0].message.content)

 Sending ReAct Prompt based request to the LLM. This might take a few moments...
 

Thought: I need to find out what primary services can be deployed with Kubernetes.
Action: Lookup('Kubernetes primary services')
Observation: Deployment, Scaling, Load Balancing.

Thought: I'm waiting for more results to provide a comprehensive answer.
Action: Lookup('use cases for Kubernetes deployment')
Observation: 
  - Web Applications
  - Microservices
  - Big Data Analytics
  - Machine Learning
  - Containerized Applications

Observation: Continuous Integration and Deployment, Configuration Management.

Thought: I want to list the primary services that can be deployed with Kubernetes.
Action: Create List('primary services of Kubernetes')
Observation:
1. Deploy
2. Scale
3. Load Balance

**Ops Focus & Python/AI for Beginners:**

* **`react_few_shot_example` (String):** Again, a multi-line string providing an example. The key here is the `Action:` and `Observation:` parts.
* **Simulated Actions:** In this notebook, `Thought: ('...')` **isn't actually running a search**. The AI is *generating text* that looks like it's planning to run a search. In a real ReAct system (often called an "AI Agent"), you'd have Python code that:
    1.  Parses the AI's "Action" (e.g., sees `Thought: I need to find the primary services provided by Kubernetes.`)
    3.  Actually *executes* that lookup (e.g., queries a database or a web search API).
    4.  Takes the result and feeds it back to the AI as the "Observation."
* **`f"""..."""` (f-string):** This is a convenient way in Python to build strings that include the values of variables. ` {react_few_shot_example}` and `{user_question_react}` insert the content of those variables into the main string.
* **`"role": "system", "content": "Answer questions using a ReAct format..."`:** This system message is crucial. It explicitly tells the AI to use the Thought-Action-Observation pattern and even suggests a tool (`Lookup`).
* **Prompt Engineering:** The way `react_prompt_for_user` is constructed is a simple example of **prompt engineering** – carefully crafting the input to the AI to guide it towards the desired output format and reasoning process. This is a critical skill when working with LLMs. You're essentially programming the AI through the instructions you give it.

**Note on the ReAct Example Style:**
* In the CoT example, the few-shot examples were part of the *system prompt*.
* In this ReAct example, the few-shot example is passed along with the *user's question*.
These are just different ways to structure the information for the AI. Experimenting with how you provide examples and instructions is part of prompt engineering. For Ops, consistency in how you format these prompts can be important for predictable AI behavior.

The output from the ReAct cell shows the AI trying to break down the problem:
1.  Identify it needs info on "Colorado Orogeny" and its "Eastern Extension."
2.  Propose `Action: Lookup(...)` for these.
3.  (Simulated) `Observation:` gets facts about the orogeny.
4.  Then it reasons it needs elevation data for specific states.
5.  Proposes another `Action: Lookup(...)` for elevations.
6.  (Simulated) `Observation:` gets elevation ranges.
7.  Finally, it synthesizes this into an `Answer:`.

This iterative process is what makes ReAct powerful for complex queries that require external knowledge or tool use.

## Chain of Thought (CoT) vs. ReAct (Reasoning + Acting): A Comparison

Here’s a summary to help you decide which approach might be better for different operational tasks.

### Chain of Thought (CoT)

* **What it is:** Purely a reasoning process. The AI "thinks aloud" step-by-step before giving an answer. It doesn't interact with external tools or live data sources on its own during this process.
* **Analogy for Ops:** Like a senior engineer verbalizing their troubleshooting steps based on their existing knowledge and documentation they've already read, without actively querying a live system during that specific explanation.

| Strengths                                                                | Limitations                                                                 |
| :----------------------------------------------------------------------- | :-------------------------------------------------------------------------- |
| Good for tasks requiring pure logical deduction (e.g., interpreting complex configuration logic, simple math, multi-step factual recall from its training data). | Cannot interact with external tools or live systems (e.g., can't check current server status, query a live database, or get real-time alerts). |
| Relatively easy to implement; you're just asking the model to explain its reasoning. | Limited if the answer depends on up-to-the-minute data or information not in its training. |
| Transparent: You can see the reasoning path, which helps in understanding and debugging the AI's output. | Not suitable for tasks requiring actions in the real world or data retrieval. |

### ReAct (Reasoning + Acting)

* **What it is:** Combines CoT-style reasoning with the ability to take **Actions** (like calling tools, performing web searches, or executing functions) and then incorporating the **Observations** (results of those actions) into its next thought process.
* **Analogy for Ops:** Like an automated runbook or a sophisticated troubleshooting script. The system reasons what to check next (`Thought`), executes a command or API call to check it (`Action`), gets the result (`Observation`), and then decides the next step based on that new information.

| Strengths                                                                 | Limitations                                                                  |
| :------------------------------------------------------------------------ | :--------------------------------------------------------------------------- |
| Excellent for building "AI Agents" that can interact with your environment (e.g., querying monitoring systems, interacting with ticketing systems, looking up internal documentation via an API). | More complex to implement. You need to build the "tools" the AI can call and the framework to handle the Action/Observation loop. |
| Enables decision-making with dynamic, real-time data.                   | Debugging can be harder if the chain of thoughts and actions becomes very long or if tools produce unexpected results. |
| You can integrate your own custom tools, scripts, and data sources (e.g., access a CMDB, query a log aggregation platform, interact with cloud provider APIs). | Requires careful design of available tools and permissions to ensure safety and prevent unintended actions. (Ops concern: security and control are paramount). |
| Highly effective for Retrieval Augmented Generation (RAG) systems, where the AI first "looks up" relevant documents before answering a question. |                                                                              |

## Summary: Which to Choose?

* **Use CoT when:**
    * The problem can be solved by reasoning alone, based on the information the AI was trained on or information provided directly in the prompt.
    * You need to understand the AI's thought process for a relatively self-contained task.
    * Examples: Summarizing a provided text in detail, solving a logic puzzle, explaining a concept based on general knowledge.
    * **Ops Example:** Asking an AI to explain the potential consequences of a specific configuration change, assuming it has been given the relevant documentation or context in the prompt.

* **Use ReAct when:**
    * The AI needs to interact with external systems, tools, or dynamic data sources to answer the question or complete a task.
    * You are building an "AI agent" that needs to perform actions.
    * The problem requires information gathering from multiple steps or sources.
    * Examples: Answering "What's the current weather in London?" (needs a weather API), "Summarize the latest critical alerts from our production environment" (needs to query a monitoring system).
    * **Ops Examples:**
        * Building an AI assistant that can check the health of a service by querying its metrics endpoint.
        * Creating an AI that can look up troubleshooting steps in an internal knowledge base when given an error code.
        * Automating the initial information gathering steps for an incident ticket by having an AI query logs, metrics, and recent deployments.

For many advanced Ops use cases where AI needs to interact with your infrastructure or internal data, **ReAct (or similar agent-based architectures)** will be the more powerful and practical approach. However, understanding CoT is a good foundation, as the "Reasoning" part of ReAct is essentially a Chain of Thought process.

This notebook has shown simple, simulated examples. Building a robust ReAct system involves:
1.  Defining the tools the AI can use.
2.  Writing Python (or other language) functions that implement those tools.
3.  Creating a loop that:
    * Sends the prompt to the AI.
    * Parses the AI's response to see if it wants to take an "Action."
    * If yes, executes the tool and gets the "Observation."
    * Feeds the "Observation" back to the AI for its next "Thought."
    * Repeats until the AI provides a final "Answer."

This area is rapidly evolving, with frameworks like LangChain and LlamaIndex providing higher-level tools to build such agentic systems. However, understanding these fundamental prompting techniques is key to effectively using and operationalizing LLMs.

In the next Module we will move onto Tool Calling - a core underlying enabler of Agentic AI